<a href="https://colab.research.google.com/github/ryouy/AKaggle/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
competition_name = "spaceship-titanic"
from google.colab import userdata
import os
os.environ['KAGGLE_USERNAME'] = 'ryohmats'
os.environ['KAGGLE_KEY'] = userdata.get('kaggle')

!pip install kaggle
!kaggle competitions download -c {competition_name}
!unzip -q /content/{competition_name}.zip -d ./
!rm -rf /content/{competition_name}.zip
!rm -rf sample_data/

!pip install ydata-profiling
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_extracted = df_train.sample(n=1000, random_state=42)
profile = ProfileReport(df_extracted, title="Training Data Report", explorative=True)
profile.to_file("train_report_extracted.html")

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 68.8MB/s]
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=968fbdfd602899a0a49ec0cca8e08a4b75737c0dad989f2f8ab42c0861d6c142
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
   

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
df_train = pd.read_csv('train.csv')
print(df_train.isnull().sum())
print(df_train.info())

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float6

In [121]:
df_train.head(-1)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687,9275_03,Europa,NaN,A/97/P,TRAPPIST-1e,30.0,False,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,True
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True


In [116]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head(-1)
data= pd.concat([df_train,df_test],axis=0,sort=False)
print(data.head(-2))
Id_data=pd.concat([df_train['PassengerId'],df_test['PassengerId']],axis=0,sort=False)
print(Id_data.head(-2))
data=data.drop(['Name'],axis=1)

data['CryoSleep'].replace({True:1,False:0},inplace=True)
data['VIP'].replace({True:1,False:0},inplace=True)
data['Transported'].replace({True:1,False:0},inplace=True)
data['Destination'].replace({'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':2},inplace=True)
data['HomePlanet'].replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)
data=data.drop(['PassengerId'],axis=1)
data=data.drop(['Cabin'],axis=1)

# 各列の欠損値を埋める
for col in data.columns:
    if col == "Transported":  # Transported列を特別処理
        data[col].fillna(-1, inplace=True)  # 欠損値を一時的に -1 に置き換え
        continue
    if data[col].dtype in ['float64', 'int64']:  # 数値型
        data[col].fillna(data[col].mean(), inplace=True)
    else:  # 文字列型やカテゴリ型
        data[col].fillna(data[col].mode()[0], inplace=True)

# Transported列が0, 1, 欠損値のみになるように確認
data['Transported'] = data['Transported'].astype(int)


     PassengerId HomePlanet CryoSleep     Cabin  Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P  TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S  TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S  TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S  TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S  TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...          ...   ...    ...   
4270     9265_01       Mars     False   D/278/S  TRAPPIST-1e  43.0  False   
4271     9266_01      Earth     False  F/1796/S  TRAPPIST-1e  40.0  False   
4272     9266_02      Earth      True  G/1496/S  TRAPPIST-1e  34.0  False   
4273     9269_01      Earth     False       NaN  TRAPPIST-1e  42.0  False   
4274     9271_01       Mars      True   D/296/P  55 Cancri e   NaN  False   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Na

In [100]:
Id_data.head(-5)

,PassengerId
0,0001_01
1,0002_01
2,0003_01
3,0003_02
4,0004_01
...,...
4267,9260_01
4268,9262_01
4269,9263_01
4270,9265_01


In [101]:
data.head(-6)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,0.0,39.0,0.00000,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,24.0,0.00000,109.0,9.0,25.0,549.0,44.0,1
2,1.0,0.0,0.0,58.0,1.00000,43.0,3576.0,0.0,6715.0,49.0,0
3,1.0,0.0,0.0,33.0,0.00000,0.0,1283.0,371.0,3329.0,193.0,0
4,0.0,0.0,0.0,16.0,0.00000,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
4266,1.0,1.0,0.0,29.0,0.00000,0.0,0.0,0.0,0.0,0.0,-1
4267,0.0,1.0,1.0,3.0,0.02154,0.0,0.0,0.0,0.0,0.0,-1
4268,0.0,0.0,1.0,20.0,0.00000,0.0,601.0,103.0,35.0,0.0,-1
4269,0.0,1.0,0.0,43.0,0.00000,0.0,0.0,0.0,0.0,0.0,-1


In [126]:
df_train = data.iloc[:8693]  # 最初の4277行
df_test = data.iloc[8693:]
Id_data_train = Id_data.iloc[:8693]
Id_data_test = Id_data.iloc[8693:]

Id_data_test.head(-8)

,PassengerId
0,0013_01
1,0018_01
2,0019_01
3,0021_01
4,0023_01
...,...
4264,9249_01
4265,9255_01
4266,9258_01
4267,9260_01


In [127]:
from re import X

X_train = df_train.drop('Transported', axis=1)
print(X_train.head(-2))
print('############')

y_train = df_train['Transported']
print(y_train.head(6))
print('############')

X_test = df_test.drop('Transported', axis=1)
print(X_test.head(6))
print('############')

y_test = df_test['Transported']
print(y_test.head(-2))
print('############')


      HomePlanet  CryoSleep  Destination   Age  VIP  RoomService  FoodCourt  \
0            1.0   0.000000          0.0  39.0  0.0          0.0        0.0   
1            0.0   0.000000          0.0  24.0  0.0        109.0        9.0   
2            1.0   0.000000          0.0  58.0  1.0         43.0     3576.0   
3            1.0   0.000000          0.0  33.0  0.0          0.0     1283.0   
4            0.0   0.000000          0.0  16.0  0.0        303.0       70.0   
...          ...        ...          ...   ...  ...          ...        ...   
8686         1.0   0.000000          0.0  32.0  0.0          1.0     1146.0   
8687         1.0   0.361848          0.0  30.0  0.0          0.0     3208.0   
8688         1.0   0.000000          1.0  41.0  1.0          0.0     6819.0   
8689         0.0   1.000000          2.0  18.0  0.0          0.0        0.0   
8690         0.0   0.000000          0.0  26.0  0.0          0.0        0.0   

      ShoppingMall     Spa  VRDeck  
0             

In [128]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [129]:
y_pred_df = pd.DataFrame(y_pred, columns=["Transported"])
y_pred_df = pd.concat([Id_data_test, y_pred_df], axis=1)
y_pred_df['Transported'] = y_pred_df['Transported'].replace({0: False, 1: True})

print(y_pred_df.head(-2))
y_pred_df.to_csv('submission.csv', index=False)

     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01         True
4        0023_01         True
...          ...          ...
4270     9265_01         True
4271     9266_01         True
4272     9266_02         True
4273     9269_01         True
4274     9271_01         True

[4275 rows x 2 columns]
